In [1]:
import gzip
import json
from pprint import pprint
from celan_app.models import Verse, Word, WordTranslation

In [2]:
 path = "data/de_wiki.gz"

In [3]:
from typing import TypedDict

class TranslationType(TypedDict):
    sense:str
    eng: str
    rus: str
    ukr: str

EntryType = list[list[TranslationType]]

def parse_wiki_item(line) -> EntryType:
    languages = {"eng": "Englisch", "rus": "Russisch", "ukr": "Ukrainisch"}
    word = line.get("word", None)
    
    parsed_word: EntryType = {}
    if not word:
        return None
    parsed_word[word] = []
    
    translations = line.get("translations", None)
    if not translations:
        return None
    
    check_translations = [False]
    
    for trans in translations:
        if trans["lang"] == languages["eng"]:
            parsed_word[word].append({"sense": trans.get("sense", ""), "eng": trans["word"]})
            check_translations.append(True)
        if trans["lang"] == languages["rus"]:
            parsed_word[word].append({"sense": trans.get("sense", ""), "rus": trans["word"]})
            check_translations.append(True)
        if trans["lang"] == languages["ukr"]:
            parsed_word[word].append({"sense": trans.get("sense", ""), "ukr": trans["word"]})
            check_translations.append(True)
    if not any(check_translations):
        return None
    return parsed_word 

In [4]:
def read_lines(path, start = 0, stop = 10):
    progress_check = range(0, 10_000_000, 50_000)
    reduced_wiki: list[dict] = []
    lines_no_trans: int = 0
    lines_with_trans: int = 0
    with gzip.open(path, "rt", encoding  = "utf-8") as f:
        for line in f:
            current_line = json.loads(line)
            parsed_line = parse_wiki_item(current_line)
            if parsed_line:
                reduced_wiki.append(parsed_line)
                lines_with_trans += 1
            else:
                lines_no_trans += 1
                
            total_lines = lines_with_trans + lines_no_trans
            if total_lines in progress_check:
                print(f"number of lines processed = {total_lines}")
    return reduced_wiki
            
            

In [5]:
reduced_wiki = read_lines(path)

number of lines processed = 50000
number of lines processed = 100000
number of lines processed = 150000
number of lines processed = 200000
number of lines processed = 250000
number of lines processed = 300000
number of lines processed = 350000
number of lines processed = 400000
number of lines processed = 450000
number of lines processed = 500000
number of lines processed = 550000
number of lines processed = 600000
number of lines processed = 650000
number of lines processed = 700000
number of lines processed = 750000
number of lines processed = 800000
number of lines processed = 850000
number of lines processed = 900000
number of lines processed = 950000
number of lines processed = 1000000
number of lines processed = 1050000
number of lines processed = 1100000
number of lines processed = 1150000
number of lines processed = 1200000
number of lines processed = 1250000


In [6]:
len(reduced_wiki)
reduced_wiki[100]["Raum"]

[{'sense': 'Physik: sich in drei Dimensionen erstreckende geometrische Größe',
  'eng': 'space'},
 {'sense': 'Physik: sich in drei Dimensionen erstreckende geometrische Größe',
  'rus': 'пространство'},
 {'sense': 'Physik: sich in drei Dimensionen erstreckende geometrische Größe',
  'ukr': 'простір'},
 {'sense': 'zum Aufenthalt oder für unterschiedliche Nutzung bestimmter, von allen Seiten umschlossener Bereich',
  'eng': 'room'},
 {'sense': 'zum Aufenthalt oder für unterschiedliche Nutzung bestimmter, von allen Seiten umschlossener Bereich',
  'rus': 'помещение'},
 {'sense': 'zum Aufenthalt oder für unterschiedliche Nutzung bestimmter, von allen Seiten umschlossener Bereich',
  'ukr': 'приміщення'},
 {'sense': 'Geografie: Gebiet ohne exakte Abgrenzung', 'eng': 'region'},
 {'sense': 'Geografie: Gebiet ohne exakte Abgrenzung', 'rus': 'район'},
 {'sense': 'Geografie: Gebiet ohne exakte Abgrenzung', 'ukr': 'район'},
 {'sense': 'Geografie: Gebiet ohne exakte Abgrenzung', 'ukr': 'область'},

In [7]:
import ipdb
def find_dupes(data):
    dupes:dict[list[int]] = {}
    uniques: dict[str, int] = {}
    for i, line in enumerate(data):
        word = list(line.keys())[0]
        if word not in uniques:
            uniques[word] = [i]
        else:
            value = uniques[word]
            value.append(i)
            dupes[word] = value
            
    print(f"number of dupes = {len(dupes)}")
    print(f"number of uniques = {len(uniques)}")
    print(f"{len(dupes) + len(uniques) == len(data)}")
    
    return dupes, uniques

In [8]:
dupes, uniques = find_dupes(reduced_wiki)

number of dupes = 2483
number of uniques = 98419
False


In [9]:
list(uniques.items())[100]

('Familie', [120])

In [10]:
list(dupes.items())[100]

('Ausdruck', [1279, 1280])

In [11]:
def turn_wiki_to_dict(data: EntryType, dupes: dict[str, list[int]]):
    progress_check = range(0, 10_000_000, 50_000)
    count = 0
    wiki: dict[list[str, str]] = {}
    for item in data:
        word = list(item.keys())[0]
        if word in dupes:
            indexes: list[int] = dupes[word]
            values: list[dict] = []
            for idx in indexes:
                values.append(data[idx][word])
            wiki[word] = values
        else:
            trans = item[word]
            wiki[word] = trans    
            
        count += 1
        if count in progress_check:
            print(count)
            
    return wiki
        

In [12]:
wiki_dict = turn_wiki_to_dict(reduced_wiki, dupes)
len(wiki_dict)

50000
100000


98419

In [13]:
word_1 = list(dupes.keys())[100]
print(word_1)
pprint(wiki_dict[word_1])

Ausdruck
[[{'eng': 'expression',
   'sense': 'Sprachforschung, Semiotik: ein Wort oder eine Gruppierung von '
            'Worten'},
  {'rus': 'выражение',
   'sense': 'Sprachforschung, Semiotik: ein Wort oder eine Gruppierung von '
            'Worten'},
  {'sense': 'Sprachforschung, Semiotik: ein Wort oder eine Gruppierung von '
            'Worten',
   'ukr': 'вираз'},
  {'sense': 'Sprachforschung, Semiotik: ein Wort oder eine Gruppierung von '
            'Worten',
   'ukr': 'вислів'},
  {'rus': 'проявление', 'sense': 'Logik: eine bestimmte Aussage'},
  {'eng': 'expression',
   'sense': 'Mathematik, Programmierung: ein syntaktisches Konstrukt'}],
 [{'eng': 'print-out',
   'sense': 'Druckwesen, EDV: ein Druckwerk als Ergebnis einer '
            'elektronisch-mechanischen Reproduktion'},
  {'eng': 'printout',
   'sense': 'Druckwesen, EDV: ein Druckwerk als Ergebnis einer '
            'elektronisch-mechanischen Reproduktion'},
  {'rus': 'распечатка',
   'sense': 'Druckwesen, EDV: ei

In [62]:
def save_file(path, file_to_save):
    with open(path, "w", encoding="utf-8") as f:
        f.write(json.dumps(file_to_save))

In [15]:
path_wiki_dict = "data/wiki_dict.json"
# save_file(path_wiki_dict)


In [16]:
def check_trans(data):
    words = Word.objects.all()
    is_trans = []
    no_trans = []
    for word in words:
        if word.lemma in wiki_dict:
            is_trans.append(word.lemma)
        else:
            no_trans.append(word.lemma)
    print(f"number of words with no translations = {len(no_trans)} ")
    print(f"number of words with translations = {len(is_trans)} ")
    return no_trans, is_trans
            

In [17]:
no_trans, is_trans = check_trans(wiki_dict)

number of words with no translations = 4849 
number of words with translations = 3540 


In [61]:
no_trans[:30]

['er|es|sie',
 'Aug',
 'daß',
 'am',
 'En',
 'Sie|sie',
 'Im',
 'Un',
 'himmeln',
 'unsre',
 'ander',
 'überm',
 'stehn',
 'zweit',
 'Ge',
 'r',
 'welch',
 'unsrer',
 'Vom',
 'beim',
 'ans',
 'unsern',
 'hinterm',
 'heut',
 'läßt',
 'IM',
 'ch',
 'aufs',
 'Am',
 'übers']

In [64]:
path_no_trans = "data/no_trans.json"
save_file(path_no_trans, no_trans)


In [26]:
wiki_dict["himmeln"]

KeyError: 'himmeln'

In [55]:
def find_word(path, word):
    progress_check = range(0, 10_000_000, 100_000)
    with gzip.open(path, "rt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            unpacked_line = json.loads(line)
            try:
                if i in progress_check:
                    print(i)
                if unpacked_line["word"] == word:
                    
                    return unpacked_line, "None"
            except:
                pass
        

In [57]:
wiki_item, trans = find_word(path, "himmeln")
pprint(trans)
pprint(wiki_item)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
'None'
{'antonyms': [{'sense_index': '1', 'word': 'missachten'},
              {'sense_index': '2', 'word': 'erden'}],
 'categories': ['Anagramm sortiert (Deutsch)',
                'Deutsch',
                'Einträge mit Endreim (Deutsch)',
                'Grundformeintrag (Deutsch)',
                'Roter Audiolink',
                'Rückläufige Wörterliste (Deutsch)',
                'Rückläufige Wörterliste Grundformeintrag (Deutsch)',
                'Verb (Deutsch)',
                'Wiktionary:Audio-Datei',
                'siehe auch'],
 'coordinate_terms': [{'sense_index': '1', 'word': 'schmachten'},
                      {'sense_index': '1', 'word': 'schwärmen'},
                      {'sense_index': '1', 'word': 'verklären'},
                      {'sense_index': '1', 'word': 'verzücken'},
                      {'sense_index': '2', 'word': 'sterben'}],
 'etymology_texts': ['strukturell: Derivation (A

In [60]:
wiki_item.keys()

dict_keys(['word', 'pos', 'pos_title', 'lang_code', 'lang', 'senses', 'sounds', 'antonyms', 'hyponyms', 'coordinate_terms', 'categories', 'etymology_texts', 'forms', 'hyphenations'])